### Config

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.tavily_search import TavilySearchResults
from pprint import pprint
import os
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

### Embedding & call llm

In [2]:

embed_model = FastEmbedEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    cache_dir="./embedding_cache" 
)

/Users/linhbk/Documents/python projects/NCKH_2024/RagAgent/agent_libs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(model_name='Llama3-8b-8192', api_key=groq_api_key)

###  Chunking text

In [4]:
import json
with open("data/vectorstore.json", "r", encoding="utf-8") as f:
    chunks_with_metadata = json.load(f)

doc_splits = [
    Document(
        page_content=chunk["text"],
        metadata={
            "chapter": chunk["metadata"]["chapter"],
            "title": chunk["metadata"]["title"],
            "date": chunk["metadata"]["date"]
        }
    ) for chunk in chunks_with_metadata
]

FileNotFoundError: [Errno 2] No such file or directory: 'data/vectorstore.json'

In [ ]:
doc_count = len(doc_splits)
doc_count

2607

## RAG

### save vector db in persist files

In [5]:
# persist_directory = './real_estate_db/luat_dat_dai'
persist_directory = './real_estate_db/vectorstore'

### chromadb

In [ ]:
# # create db
# vectorstore_created = Chroma.from_documents(documents=doc_splits,
#                                     embedding=embed_model,
#                                     persist_directory=persist_directory,
#                                     collection_name="vectorstore")
# vectorstore_created.persist()

In [6]:
# call from existed db
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed_model, collection_name="vectorstore")
# vectorstore.get()

/var/folders/cc/64gffy7j1_79tlsdxkhtcjg00000gp/T/ipykernel_97340/1442968809.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed_model, collection_name="vectorstore")


In [7]:
print("Number of stored documents:", vectorstore._collection.count())
# print("First document:", doc_splits[-1].page_content if doc_splits else "No documents found!")
print("Files in persistence directory:", os.listdir(persist_directory))

Number of stored documents: 2607
Files in persistence directory: ['1aead4fb-bcb3-496d-86d1-dda9b04741f6', 'chroma.sqlite3']


In [8]:
from chromadb import PersistentClient
client = PersistentClient(path=persist_directory)
collections = client.list_collections()
print("Available collections:", collections)


Available collections: ['vectorstore']


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever_lambda = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))

In [10]:
# from langchain.schema import Document
# from langchain_core.runnables import RunnableLambda

# # convert chunks to document
# def format_text_chunks(text_chunks):
#     return [Document(page_content=chunk) for chunk in text_chunks]

# retriever_lambda = RunnableLambda(lambda x: format_text_chunks(retriever.get_relevant_documents(x["question"])))

## Component prompts

In [ ]:
filter_prompt = PromptTemplate(
    template="""
    You are an AI-based information filter responsible for categorizing user input questions.
    Your mission is to return a binary choice "yes" or "no" indicating whether the question is related to sensitive topics.
    Sensitive topics include hate-speech, sexuality, politics, historical, violence, religion.
    
    **IMPORTANT**: Your response **MUST** be a valid JSON object with a single key "score" and a value of "yes" or "no". 
    **NOTE**: If topic is related to vietnamese laws, its not the sensitive topic even refer to senstive topics. 
    **NOTE**: Some hate-speech often wrote in short cut ("đcm", "mm", "vl") 
    **DO NOT** include any other text or explanation.
    
    For example: 
    Input: "ai là người lãnh đạo đảng?"
    Output: {{"score": "yes"}}

    Input: "việt tân là ai?"
    Output: {{"score": "yes"}}
    
    Input: "đcmm"
    Output: {{"score": "yes"}}
    
    Input: "luật việt nam là như nào?"
    Output: {{"score": "no"}}
    
    Input: "tội hiếp dâm và giết người bị phán bao nhiêu năm tù?"
    Output: {{"score": "no"}}
    
    Question need to filtered: {question}
    """,
    input_variables=["question"]
)
filter_chain = (filter_prompt | llm | JsonOutputParser())

# test
question = "“Người sử dụng đất” được hiểu như thế nào theo quy định của Luật Đất đai năm 2024?"
filter_result = filter_chain.invoke({"question": question})
print(filter_result)

{'score': 'no'}


### Router

In [12]:
router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on real estate laws in Vietnam. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)
start = time.time()
question_router = router_prompt | llm | JsonOutputParser()

# test
print(question_router.invoke({"question": question}))
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.24171710014343262


### Generator

In [ ]:
from langchain_core.runnables import RunnableLambda

qa_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just refuse answer in polite and friendly. 
    Answer question in detailed, make sure references vietnamese's law that prove for your answer. 
    For example:
    Question: Phạm vi điều chỉnh và đối tượng áp dụng Luật Đất đai năm 2024 là gì?
    Answer: 
    Điều 1. Phạm vi điều chỉnh
    Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của công dân, người sử dụng đất đối với đất đai thuộc lãnh thổ của nước Cộng hòa xã hội chủ nghĩa Việt Nam.
    Điều 2. Đối tượng áp dụng
    1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất đai, thực hiện nhiệm vụ thống nhất quản lý nhà nước về đất đai.
    2. Người sử dụng đất.
    3. Các đối tượng khác có liên quan đến việc quản lý, sử dụng đất đai.'
    
    **IMPORTANT**: Your response **MUST** be in Vietnamese, the tone have to professional and polite
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Chain
start = time.time()
rag_chain = (
    {"question": lambda x: x["question"], "context": retriever_lambda}
    | qa_prompt
    | llm
    | StrOutputParser()
)

# test
# question = "luật nhà ở 2024"
response = rag_chain.invoke({"question": question})
print(response)

/var/folders/cc/64gffy7j1_79tlsdxkhtcjg00000gp/T/ipykernel_97340/2309398310.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_lambda = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))


Theo quy định của Luật Đất đai năm 2024, người sử dụng đất được hiểu là các tổ chức, cá nhân có quyền sử dụng đất, bao gồm:

* Chủ sở hữu đất đai: là người có quyền sở hữu đất đai theo quy định của pháp luật, bao gồm cá nhân, hộ gia đình, tổ chức và người Việt Nam có tư cách pháp nhân.
* Người được giao quyền sử dụng đất: là người được Nhà nước giao quyền sử dụng đất, bao gồm người được giao quyền sử dụng đất theo quyết định của cơ quan nhà nước và người được giao quyền sử dụng đất theo hợp đồng.
* Người được thừa kế quyền sử dụng đất: là người được thừa kế quyền sử dụng đất theo quy định của pháp luật.

Theo Điều 28 của Luật Đất đai năm 2024, người sử dụng đất phải có quyền sử dụng đất và chịu các nghĩa vụ quản lý, sử dụng đất đai theo quy định của pháp luật.

Theo Điều 40 của Nghị định số 96/2024/NĐ-CP, người sử dụng đất phải cung cấp thông tin về quyền sử dụng đất, bao gồm quyền sử dụng đất, hạn chế về quyền sử dụng đất (nếu có), tiến độ thực hiện và các nội dung khác.

Căn cứ vào c

### Retriever

In [14]:
retrieval_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. \n
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)
start = time.time()
retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

# test
docs = retriever.invoke(question)
end = time.time()

for i, doc in enumerate(docs):
    doc_txt = doc.page_content
    print(f"doc {i + 1} content: {doc_txt}")
    doc_grader = retrieval_grader.invoke({"question": question, "document": doc_txt})
    print(f"retriever {i + 1} grade: {doc_grader}")


doc 1 content: Điều 28.Nhận quyền sử dụng đất
1
retriever 1 grade: {'score': 'yes'}
doc 2 content: - Những hạn chế về quyền sử dụng đất (nếu có): ..........................................................
2
retriever 2 grade: {'score': 'yes'}
doc 3 content: quyền sử dụng đất thì không cần mô tả thông tin này)
- Tiến độ thực hiện: .....................................................................................................
- Các nội dung khác: ...................................................................................................
điều 2
retriever 3 grade: {'score': 'no'}


### Hallucination

In [15]:
hallucination_grader_prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = hallucination_grader_prompt | llm | JsonOutputParser()

# test
hallucination_grader_response = hallucination_grader.invoke({"documents": docs, "generation": response})
end = time.time()
print(f"The time required to generate response by the generation chain in seconds:{end - start}")
print(hallucination_grader_response)

The time required to generate response by the generation chain in seconds:0.34575486183166504
{'score': 'yes'}


### ANswer grader

In [16]:
answer_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = answer_grader_prompt | llm | JsonOutputParser()

# test
answer_grader_response = answer_grader.invoke({"question": question,"generation": response})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.35233521461486816
{'score': 'yes'}


### Websearch tool

In [17]:
tavily_api_key = os.environ['TAVILY_API_KEY']
web_search_tool = TavilySearchResults(k=3)


Single test web_search_tool

In [18]:
import os
from langchain.tools import TavilySearchResults

def search_web(query: str, k: int = 3):
    """
    Searches the web using Tavily API for the given query.
    
    Parameters:
    - query (str): The search query.
    - k (int): Number of search results to return (default is 3).
    
    Returns:
    - list: A list of search result snippets.
    """
    tavily_api_key = os.environ['TAVILY_API_KEY']
    web_search_tool = TavilySearchResults(k=k)
    
    try:
        results = web_search_tool.run(query)
        return results
    except Exception as e:
        print(f"Error while searching: {e}")
        return []

query = "TÌnh hình bất đọngo sản 2024"
search_results = search_web(query)
print(search_results)


[{'title': 'Dự báo bất động sản 2024 - CafeLand.Vn', 'url': 'https://cafeland.vn/chu-de-nong/du-bao-bat-dong-san-2024-3101/', 'content': 'Theo dự báo của chuyên gia, năm 2024, ngành bất động sản sẽ đón nhận sự quay trở lại của khoảng 30%-40% môi giới bất động sản. Sự phục hồi rõ nét của thị trường sẽ được thể hiện từ cuối quý 3.2024....\n\nNgành bất động sản trong năm 2024 sẽ diễn biến theo xu hướng nào?\n\nTái cấu trúc nguồn vốn, M&A dự án và tập trung vào phân khúc trung cấp dự báo sẽ là những xu hướng chính của ngành Bất động sản trong năm 2024. Cụ thể ra sao, các bạn cùng theo dõi tiếp video này nhé.... [...] Thị trường bất động sản được dự báo sẽ có sự chuyển mình trong năm 2024, với hai phân khúc dẫn dắt. Tuy nhiên, khó khăn vẫn đang tiếp diễn, đòi hỏi doanh nghiệp phải nhanh nhạy nắm bắt thời cơ, dồn lực vào các dự án khả thi đồng thời có chính sách li...\n\nKịch bản xấu và nguy hiểm cho thị trường bất động sản vẫn có thể xảy ra [...] Mặc dù tình hình kinh tế ch

## LangGraphh

In [19]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]
    iterations: int

### Nodes

In [20]:
def parse_question(state):
    question = state["question"]
    return {"question": question}

In [21]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    try:
        documents = retriever.invoke(question)
    except Exception as e:
        print(f"Error in retrieval: {e}")
        documents = []
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    iterations = state.get("iterations", 0) + 1
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "iterations": iterations}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])
    iterations = state.get("iterations", 0) + 1

    # Web search
    docs = web_search_tool.run({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question, "iterations": iterations}

### Condition edges

In [22]:
def filter_question(state):
    """
    Filter the question for sensitive topics
    
    Args:
        state (dict): The current graph state
    
    Returns:
        dict: Updated state with decision to end or continue
    """
    print("---FILTER QUESTION---")
    question = state["question"]
    filter_result = filter_chain.invoke({"question": question})
    if filter_result["score"] == "yes":
        print("---SENSITIVE TOPIC DETECTED---")
        return "yes"
    else:
        print("---QUESTION PASSED FILTER---")
        return "no"

In [23]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


In [24]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [25]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    iterations = state.get("iterations", 0)
    print(f"number of iteration at this hallucination term: {iterations}")
    
    if iterations >= 5:
        print("GETTING MAX ATTEMPTS")
        return "end"

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

### Add node

In [26]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("route_question", route_question)
workflow.add_node("parse_question", parse_question)
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

### Entry & End points

In [27]:
workflow.set_conditional_entry_point(
    filter_question,
    {
        "yes": END,
        "no": "parse_question"
    },
)

workflow.add_conditional_edges(
    "parse_question",
    route_question,     
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
        "end": END
    },
)

In [28]:
app = workflow.compile()

## Test

In [29]:
inputs = {"question": "what is my name?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

        if key == "filter_question" and value == "yes":
            pprint("xin loi toi khong the tra loi cau hoi nay")
            break

    if "generation" in value:
        pprint(value["generation"])

---FILTER QUESTION---
---QUESTION PASSED FILTER---
---ROUTE QUESTION---
what is my name?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
'Finished running: parse_question:'
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
number of iteration at this hallucination term: 2
'---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---'
'Finished running: generate:'
("I'm happy to help! However, I must politely point out that the context you "
 'provided does not include any information about your name. The documents you '
 'shared are related to various Vietnamese laws, but they do not contain any '
 'personal information about you or your name.\n'
 '\n'
 "As an assistant, I'm designed to provide accurate and reliable information "
 'based on the context provided. In this case, I do not have any information '
 'about your name, and I cannot provide an answer to this question.\n'
 '\n'
 "If you could provide more contex

### Deploy model to gradio

In [38]:
def answer(input):
    question  = {"question": input}
    for output in app.stream(question):
        for key, value in output.items():
            if key == "filter_question" and value == "yes":
                return "Xin lỗi, mình không thể trả lời câu hỏi này vì nó chứa nội dung liên quan đến chủ đề nhạy cảm."

        if "generation" in value:
            return value["generation"]
    return "Xin lỗi, mình không có câu trả lời cho câu hỏi của bạn"

In [40]:
input = "mẹ m"
print(answer(input))

---FILTER QUESTION---
---SENSITIVE TOPIC DETECTED---
Xin lỗi, mình không có câu trả lời cho câu hỏi của bạn
